In [1]:
import os
import sys

import pandas as pd
import dotenv
import mlflow
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import plotly.graph_objects as go
from huggingface_hub import login

sys.path.append('..')

from utils import calculate_metrics

mlflow.set_tracking_uri('http://127.0.0.1:5000')
dotenv.load_dotenv('../../.env')

token = os.environ["HF_TOKEN"]
login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
data_dir = '../../data/panama-electricity-load-forecasting/processed/'

train_df = pd.read_parquet(os.path.join(data_dir, 'train.parquet'))
test_df = pd.read_parquet(os.path.join(data_dir, 'test.parquet'))

In [3]:
train_df_chronos = train_df[['datetime', 'nat_demand']].rename(columns={'datetime': 'timestamp', 'nat_demand': 'target'}).copy()
train_df_chronos['item_id'] = 0

test_df_chronos = test_df[['datetime', 'nat_demand']].rename(columns={'datetime': 'timestamp', 'nat_demand': 'target'}).copy()
test_df_chronos['item_id'] = 0

train_data = TimeSeriesDataFrame.from_data_frame(train_df_chronos)
test_data = TimeSeriesDataFrame.from_data_frame(test_df_chronos)

In [4]:
predictor = TimeSeriesPredictor(prediction_length=744).fit(
    train_data,
    hyperparameters={
        "Chronos": {"model_path": "autogluon/chronos-bolt-base"},
    },
)

predictions = predictor.predict(train_data)

Beginning AutoGluon training...
AutoGluon will save models to '/home/nikita/projects/time_series_analysis/code_dir/Chronos/AutogluonModels/ag-20250330_184545'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #53~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jan 15 19:18:46 UTC 2
CPU Count:          12
GPU Count:          1
Memory Avail:       20.18 GB / 30.95 GB (65.2%)
Disk Space Avail:   188.18 GB / 233.67 GB (80.5%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'autogluon/chronos-bolt-base'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 744,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time 

In [5]:


train_size = 500
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=train_df["datetime"][-train_size:],
        y=train_df["nat_demand"][-train_size:],
        name="Train",
        line=dict(color="#6495ED"),
    )
)
fig.add_trace(
    go.Scatter(
        x=test_df["datetime"],
        y=test_df["nat_demand"],
        name="Test",
        line=dict(color="#50C878"),
    )
)
fig.add_trace(
    go.Scatter(
        x=test_df["datetime"],
        y=predictions["mean"],
        name="Chronos Forecast (mean)",
        line=dict(color="#D70040", dash="dot"),
    )
)

fig.add_trace(go.Scatter(
    x=test_df['datetime'],
    y=predictions['0.9'],
    mode='lines',
    line=dict(width=0),
    name='90th percentile',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=test_df['datetime'],
    y=predictions['0.1'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 127, 14, 0.6)',
    line=dict(width=0),
    name='Chronos Confidence Interval (0.1-0.9)'
))

fig.update_layout(
    title="Target value",
    xaxis_title="Date",
    yaxis_title="National Demand",
    template="plotly_white",
    width=1_400,
    height=500,
)
fig.show()

In [8]:
y_true = test_df['nat_demand']
y_pred = predictions['mean']

metrics = calculate_metrics(y_true=y_true, y_pred=y_pred)

exp_name = 'Time_Series_Forecasting'
run_name = 'Chronos_ZeroShot_SingleSeries'
mlflow.set_experiment(exp_name)
with mlflow.start_run(run_name=run_name):
    mlflow.log_metrics(metrics)

🏃 View run Chronos_ZeroShot_SingleSeries at: http://127.0.0.1:5000/#/experiments/185045746886025740/runs/bf9caea73804417f87ad136d67efbda7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/185045746886025740
